# Import section

In [1]:
from stage2 import *
import numpy as np
from astropy.io.votable import parse

# Getting list of galaxies

In [29]:
#input file
galsLocation = "SelectedGals_outputFolders"
galsZfile    = "SelectedGals"
galsLSFfile  = "SelectedGals_clean_o2"

#get gals location
listGals     = np.genfromtxt(galsLocation, dtype="str")

#keep folders whereto the input file will be written
galsFolders  = [gal.split("o2")[0] for gal in listGals]
#keep gal name for the input file
galNames     = [gal.split("o2/")[1] for gal in listGals]

#get gals redshift
galsZ        = np.genfromtxt(galsZfile, dtype='float', usecols=1)
print("Same size ?", len(galsZ)==len(galNames))

#get gals lsf FWHM (in km/s)
galsLSF      = np.genfromtxt(galsLSFfile, dtype='float', usecols=1)
print("Same size ?", len(galsLSF)==len(galNames))

Same size ? True
Same size ? True


# Recover PA (for some galaxies) and b/a values from the .vot file

## Get full data from the .vot file

In [38]:
votFile = "../outputs/selected_field_gals.vot"

#checking that it is a vot file and retrieve data
if is_VOtable(votFile):
    full = parse(votFile).get_first_table().array
    
#Get gals number
galsNum = [int(num.split('_')[-2]) for num in listGals]

#Get gals group and convert it into the same format as that in the .vot catalogue
groupList = [name.split('MUSE/')[1].split('/o2')[0].split('_s')[0].split('CGr')[1] for name in listGals]
for pos, gr in enumerate(groupList):
    if gr == "84-N":
        groupList[pos] = 84.1
    elif gr in ("84_d", "34_d", "79_d", "30_d", "84_bs", "34_bs", "79_bs", "30_bs"):
        groupList[pos] = float(gr.split("_")[0])
    elif gr in ('32-M1_d', '32-M2_d', '32-M3_d'):
        groupList[pos] = 32.0
    else:
        groupList[pos] = float(gr)

print(groupList)

The file ../outputs/selected_field_gals.vot is a VOtable, right ? True
[84.1, 84.1, 84.1, 84.0, 84.0, 84.0, 84.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 114.0, 114.0, 114.0, 114.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 26.0, 26.0, 26.0, 26.0, 26.0, 26.0, 26.0, 26.0, 26.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 51.0, 51.0, 61.0, 61.0, 61.0, 61.0, 61.0, 61.0, 61.0, 61.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 84.1, 84.0, 34.0, 34.0, 79.0, 32.0, 32.0, 32.0, 114.0, 23.0, 28.0, 51.0, 51.0, 51.0, 61.0, 30.0, 30.0, 30.0, 30.0]


## Gals in CGr84 have the wrong MUSE Id in their folder name so we need to associate them with the correct one using another file

In [47]:
matchingFile = '../data/catalogues/CGR84-83_FD+North_zcatalog_withLaigle+16_withFAST_withnewPLATEFIT_totalflux_withnewz_jan19_withFOF_withGALFIT_withGALKIN_jan19.vot'
if is_VOtable(matchingFile):
    tmpCOSMOSID   = parse(matchingFile).get_first_table().array['COSMOS_Group_Number']
    m             = tmpCOSMOSID==84.0
    #the corresponding MUSE id in the .vot file with the morphological information
    CatalogMUSEID = parse(matchingFile).get_first_table().array['ID'][m]
    #the MUSE id found in the folders
    FolderMUSEID  = parse(matchingFile).get_first_table().array['ID_in_MUSE_Field'][m]
else:
    raise Exception("The file", matchingFile, "could not be found")

newGalsNums = galsNum[:]
for pos in np.where(np.asarray(groupList)==84.0)[0]:
    newGalsNums[pos] = CatalogMUSEID[FolderMUSEID==galsNum[pos]][0]
print(np.asarray(newGalsNums) == np.asarray(galsNum))

The file ../data/catalogues/CGR84-83_FD+North_zcatalog_withLaigle+16_withFAST_withnewPLATEFIT_totalflux_withnewz_jan19_withFOF_withGALFIT_withGALKIN_jan19.vot is a VOtable, right ? True
[ True  True  True False False False False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]


## Recover PA and inclination

In [108]:
#complete morphological info from the catalogue
fullPA    = full['PA_GIM2D_ZURICH']
fullb_a   = full['Axial_Ratio_CASSATA']
fullGrNb  = full['COSMOS_Group_Number']
fullGalNb = full['ID']

#we find the PA and the inclination for the desired galaxies by matching their group and galaxy numbers
PAlist    = []
incList   = []
for gal, gr in zip(newGalsNums, groupList):
    
    #we get the position in the full array where the galaxy lies
    pos = np.where(np.logical_and(fullGrNb == gr, fullGalNb == gal))[0]
    #we get the corresponding PA
    tmp = fullPA[pos]
    
    #if the value is masked, i.e. missing, we put by default 60 degrees
    if tmp.mask[0] :
        tmp = 60.0
    else:
        tmp = tmp[0]
        
    #if the value is -9999.0 (galaxy found in Cassata catalogue but not in Zurich), we put by default 60 degrees
    if tmp == -9999.0:
        tmp = 60.0
    
    #changing PA definition from [0, 180]° to [-90, +90]°
    PAlist.append(tmp-90)
    
    #we get the corresponding b/a, compute the inclination (between 0° and 90°) from it and add it to the list
    tmp = fullb_a[pos][0]
    incList.append(np.arccos(tmp)*180/np.pi)
    
PAlist  = np.asarray(PAlist)
incList = np.asarray(incList)

# Creating a general class to gather group information into one object

In [84]:
class groupStructure:
    
    def __init__(self, groupName, listGals, redshifts, lsf, wavelengthOfObservation=3729, PA=0, inclination=-30):
        '''
        Intialisation of the group structure
        
        Mandatory inputs
        ----------------
        groupName : string
            the name of the group
        listGals : list of strings
            the path to the gals folders
        lsf : list of floats
            the line spread function FWHM of the galaxies in km/s    
        redshifts : list of floats
            the redshifts of the galaxies
            
        Optional inputs
        ---------------
        inclination : list of floats
            the inclination of the galaxies on the sky (between 0° and +90°)
        PA : list of integers/floats
            the position angle of the galaxies (between -90° and +90°)
        wavelengthOfObservation : list of integers/floats
            the wavelength at which we observed the galaxies (default is OII, must be given in Angstroms)
        '''

        #galaxies folders and names
        self.__ln        = len(listGals)
        self.group       = groupName
        self.galsNames   = [gal.split("o2/")[1] for gal in listGals]
        self.galsFolders = [gal.split("o2")[0] for gal in listGals]
        
        #centre positions
        self.XcenPos     = [15.0]*self.__ln
        self.YcenPos     = [15.0]*self.__ln
        
        #redshift
        self.z           = [z for z in redshifts]
        
        #lsf FWHM
        self.lsf         = [l for l in lsf]
        
        #define default wavelength of observation for this group (OII at the group redshift in Angstrom)
        try:
            len(wavelengthOfObservation)
        except TypeError:
            wavelengthOfObservation = [wavelengthOfObservation]*self.__ln
        self.wv          = wavelengthOfObservation
        self.wv          = [wv*(1+z) for wv, z in zip(self.wv, self.z)]
        
        #generate PSF for each galaxy
        self.psf         = computeGroupFWHM(self.wv, [self.group]*self.__ln, verbose=False)
        
        #set default PA
        try:
            len(PA)
        except TypeError:
            PA  = [PA]*self.__ln
        self.PA = PA
        
        #set default inclination
        try:
            len(inclination)
        except TypeError:
            inclination  = [inclination]*self.__ln
        self.inclination = inclination
        
    def updateCenPos(self, Xpos, Ypos):
        '''
        Update galaxies centre X and Y position (in pixels in the MUSE image)
        
        Input
        -----
        Xpos : list of floats
            the X position of the galaxies centres
        Ypos : list of floats
            the Y position of the galaxies centres
        '''
        
        if not ( (len(XPos) == len(Ypos)) == self.__ln ):
            raise Exception("Given X and Y positions have length %d but data has length %d" %(len(XPos), self.__ln))
        self.XcenPos = [i for i in Xpos]
        self.YcenPos = [i for i in Ypos]
        
    def updatePSF(self, wavelength, verbose=False):
        '''
        Update the PSF for each galaxy using the given wavelength of observation (not rest-frame)
        
        Input
        -----
        verbose : boolean
            whether to print the computed value on screen or not
        wavelength : list of integers
            the wavelength of observation equals to (1+z) times the rest-frame wavelength
        '''
        
        if len(wavelength) != self.__ln:
            raise Exception("Given wavelength list has length %d but data has length %d" %(len(wavelength), self.__ln))
        self.psf = computeGroupFWHM(wavelength, [self.group]*self.__ln, verbose=verbose)
        
    def updatePA(self, PA):
        '''
        Update the PA of the galaxies
        
        Input
        -----
        PA : list of inetgers/floats
            the PA of the galaxies between -90° and +90°
        '''
        if len(PA) != self.__ln:
            raise Exception("Given PA list has length %d but data has length %d" %(len(PA), self.__ln))
        self.PA = PA
    
    def updateInclination(self, inclination):
        '''
         Update the inclination of the galaxies
        
        Input
        -----
        inclination : list of floats
            the inclination of the galaxies (between the line of sight and the normal of the galactic plane) between 0° and 90°
        '''
        if len(inclination) != self.__ln:
            raise Exception("Given inclination list has length %d but data has length %d" %(len(inclination), self.__ln))
        self.inclination = inclination



# Constructing the group objects

In [112]:
def returnCorrectVals(someList, GrNames, THEname):
    '''
    Returns a list of values for some group by specifying its name
    
    Input
    -----
    GrNames : list of string
        the group names corresponding to the galaxies
    someList : list of data
        a list with the same strcture as GrNames
    THEname : string
        the name of the group we want to keep the data
    '''
    
    return someList[np.where(np.asarray(GrNames) == THEname)]

#first we create a dict with the indices of galaxies belonging to each group
names    = np.asarray([i.split("MUSE/")[1].split("/o2")[0].split("_s")[0] for i in listGals])
maskDict = {}
for gr in names:
    maskDict.update({gr : np.where(names==gr)})

#then we create a dictionnary combining all the group objects
GroupsDict = {}
for gr in names:
    GroupsDict.update({gr : groupStructure(gr.split('CGr')[1], 
                                           listGals[maskDict[gr][0]],
                                           galsZ[maskDict[gr][0]],
                                           galsLSF[maskDict[gr][0]],
                                           PA = PAlist[maskDict[gr][0]],
                                           inclination = incList[maskDict[gr][0]])})

SystemExit: Given group CGr84-N is not correct. Possible values are dict_keys(['23', '26', '28', '32-M1', '32-M2', '32-M3', '51', '61', '79', '84-N', '30_d', '30_bs', '84', '34_d', '34_bs', '114'])